In [21]:
from confluent_kafka import Consumer, KafkaError
import json

settings = {
    'bootstrap.servers': 'kafka-1:19092, kafka-2:19093, kafka-3:19094',
    'group.id': 'sensorGroup-1',
    'client.id': 'client-1',
    'enable.auto.commit': True,
    'session.timeout.ms': 6000,
    'default.topic.config': {'auto.offset.reset': 'smallest'}
}

c = Consumer(settings)

c.subscribe(['sensor'])

try:
    while True:
        msg = c.poll(0.1)
        if msg is None:
            continue
        elif not msg.error():
            # print('Received message: {0}'.format(msg.value()))
            sensorData = json.loads(msg.value())
            for sensor in sensorData['sensors']:
                if sensor['title'] == 'PM10':
                    print(sensor['lastMeasurement']['value'], sensor['lastMeasurement']['unit'])
            
        elif msg.error().code() == KafkaError._PARTITION_EOF:
            print('End of partition reached {0}/{1}'
                  .format(msg.topic(), msg.partition()))
        else:
            print('Error occured: {0}'.format(msg.error().str()))

except KeyboardInterrupt:
    pass

finally:
    c.close()

KeyError: 'unit'